In [1]:
!pip install nest_asyncio
!pip install telethon

import asyncio
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
import pandas as pd
import nest_asyncio
from google.colab import files

# Apply nest_asyncio patch
nest_asyncio.apply()

# Telegram credentials
username = 'Gokul24'  # Your Telegram account username (e.g., 'abc123')
phone = '+919629261738'  # Your Telegram account phone number with country code
api_id = '20296869'  # Your API ID (from https://my.telegram.org/apps)
api_hash = '0a35e50b563051e8fa11e164f98e8875'  # Your API Hash (from https://my.telegram.org/apps)

# Initialize Telegram client
client = TelegramClient('session_name', api_id, api_hash)

async def list_channels():
    """
    List all channels and groups the user is part of.
    """
    async with client:
        if not await client.is_user_authorized():
            await client.send_code_request(phone)
            await client.sign_in(phone, input('Enter the code sent to your Telegram: '))
            try:
                await client.sign_in(password=input('Enter your 2FA password (if applicable): '))
            except SessionPasswordNeededError:
                print("Two-factor authentication password needed.")
                raise

        print("\nFetching your channels and groups...\n")
        dialogs = await client.get_dialogs()
        for dialog in dialogs:
            if dialog.is_channel:
                print(f"Name: {dialog.name}, ID: {dialog.entity.id}")

async def scrape_messages(channel_identifier):
    """
    Scrape messages from a public or private channel.

    Args:
        channel_identifier: The username (string) or ID (integer) of the channel.
    """
    async with client:
        if not await client.is_user_authorized():
            await client.send_code_request(phone)
            await client.sign_in(phone, input('Enter the code sent to your Telegram: '))
            try:
                await client.sign_in(password=input('Enter your 2FA password (if applicable): '))
            except SessionPasswordNeededError:
                print("Two-factor authentication password needed.")
                raise

        print(f"Scraping messages from channel: {channel_identifier}...\n")
        all_messages = []

        # Fetch messages from the specified channel
        async for message in client.iter_messages(channel_identifier, limit=100):  # Adjust limit as needed
            all_messages.append({
                'message_id': message.id,
                'date': message.date,
                'text': message.text or '',
                'views': message.views if hasattr(message, 'views') else None,
                'forwards': message.forwards if hasattr(message, 'forwards') else None,
                'replies': message.replies.replies if message.replies else None,
                'media': 'Yes' if message.media else 'No'
            })

        # Convert collected messages to a Pandas DataFrame
        df = pd.DataFrame(all_messages)

        # Convert the 'date' column to datetime format
        df['date'] = pd.to_datetime(df['date'])

        # Display basic info about scraped data
        print("\nScraping complete. Here is a summary of the collected data:")
        print(df.info())

        # Save the data to a CSV file and download
        output_file = 'telegram_scraped_data.csv'
        df.to_csv(output_file, encoding='utf-8', index=False)
        files.download(output_file)

        print("\nScraped data successfully saved and downloaded.")

# Main choice loop
while True:
    print("\n1. List all your channels/groups.")
    print("2. Scrape messages from a channel.")
    print("3. Exit.")
    choice = input("Enter your choice (1, 2, or 3): ")

    if choice == '1':
        print("\nListing all your channels and groups...\n")
        asyncio.get_event_loop().run_until_complete(list_channels())
    elif choice == '2':
        channel_identifier = input("Enter the username (without '@') or channel ID: ")
        try:
            # Convert to integer if it's a numeric ID
            channel_identifier = int(channel_identifier)
        except ValueError:
            pass  # Keep as string if it's a username
        asyncio.get_event_loop().run_until_complete(scrape_messages(channel_identifier))
    elif choice == '3':
        print("Exiting the program. Goodbye!")
        break
    else:
        print("Invalid choice. Please try again.")


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.2/702.2 kB 9.9 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=bb5783a97baf9bcd39fe5079a1f80b3eab5001b0832bedf76ba1592b3cd03bbd
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built pyaes

1. List all your channels/groups.
2. Scrape messages from a channel.
3. Exit.
Enter your choice (1, 2, or 3): 1

Listing all your channels and groups...

Please enter your phone (or bot token): +91 9629261738
Please enter the code you received: 46959
Signed in successfully as Gokul; remember to not break the ToS or you will risk an account ban!

Fetching your channels and groups...

Name: PENNANT TRADERS 🚩, ID: 1265516238
Name: NIFTY 50 BANKNIFTY DOMINATOR, ID: 1309531950
Name: BANKNIFTY NIFTY INTRADAY Trading, ID: 1703062664
Name: _.codedevotee, ID: 2175087629
Name: Intraday Banknifty Stock

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Scraped data successfully saved and downloaded.

1. List all your channels/groups.
2. Scrape messages from a channel.
3. Exit.
Enter your choice (1, 2, or 3): 3
Exiting the program. Goodbye!
